# Add data from GeoNames

**[Work in progress]**

This notebook adds latitude, longitude, elevation, and population data from GeoNames to Country, Admin1, Admin2, and City .csv files for ingestion into the Knowledge Graph.

Data source: [GeoNames.org](https://download.geonames.org/export/dump/)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from pathlib import Path
from io import BytesIO
import requests
from zipfile import ZipFile
import pandas as pd

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_HOME = Path(os.getenv('NEO4J_HOME'))
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3


### Download data from GeoNames

In [4]:
country_url = 'https://download.geonames.org/export/dump/allCountries.zip'

In [5]:
content = requests.get(country_url)
zf = ZipFile(BytesIO(content.content))

for item in zf.namelist():
    print("File in zip: "+  item)

File in zip: allCountries.txt


In [6]:
columns = ['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 
           'feature class', 'feature code', 'country code', 'cc2', 'admin1 code', 
           'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation', 'dem',
           'timezone', 'modification date']

In [7]:
df = pd.read_csv(zf.open('allCountries.txt'), names=columns, sep='\t', dtype='str', header=0)
df.fillna('', inplace=True)

#### Keep only feature class A (country, state, region,...) and P(city, village,...)

In [8]:
df = df[(df['feature class'] == 'A') | (df['feature class'] == 'P')]

In [9]:
# Missing population data are sometimes represented as zero, these will be ignored.
df['population'] = df['population'].str.replace('0', '')

In [10]:
df.rename(columns={'geonameid': 'geonameId'}, inplace=True)

In [11]:
df = df[['geonameId', 'latitude', 'longitude', 'elevation', 'population']]

#### Add latitude, longitude, and elevation data for countries
Note, elevation data are excluded here since they are missing for countries.

In [12]:
dfc = df[['geonameId', 'latitude', 'longitude', 'population']]

In [13]:
dfc.head()

,geonameId,latitude,longitude,population
9,3038816,42.55327,1.48736,
24,3038832,42.53176,1.56654,
91,3038899,42.48597,1.4891,
179,3038987,42.56461,1.52757,
191,3038999,42.57688,1.66769,62


In [14]:
country = pd.read_csv(NEO4J_HOME / "import/00e-GeoNamesCountry.csv", dtype='str')

In [15]:
country = pd.merge(country, dfc, on='geonameId', how='left')
country.fillna('', inplace=True)

Fix id and iso code for Namibia (NA is interpreted as NaN in Pandas!)

In [16]:
# reset the id and iso code for Namibi.
index = country.query("iso3 == 'NAM'").index
country.at[index, 'iso'] = 'NA'
country.at[index, 'id'] = 'NA'

In [17]:
country.to_csv(NEO4J_HOME / "import/00e-GeoNamesCountry.csv", index=False)

In [18]:
country.head(300)

,id,name,iso,iso3,areaSqKm,geonameId,latitude_x,longitude_x,population_x,latitude_y,longitude_y,population_y
0,AD,Andorra,AD,AND,468,,,,,,,
1,AE,United Arab Emirates,AE,ARE,82880,290557,23.75,54.5,963959,23.75,54.5,963959
2,AF,Afghanistan,AF,AFG,647500,1149361,33,66,37172386,33,66,37172386
3,AG,Antigua and Barbuda,AG,ATG,443,3576396,17.05,-61.8,96286,17.05,-61.8,96286
4,AI,Anguilla,AI,AIA,102,3573511,18.21667,-63.05,13254,18.21667,-63.05,13254
5,AL,Albania,AL,ALB,28748,,,,,,,
6,AM,Armenia,AM,ARM,29800,,,,,,,
7,AO,Angola,AO,AGO,1246700,3351879,-12.5,18.5,389762,-12.5,18.5,389762
8,AQ,Antarctica,AQ,ATA,14000000,6697173,-82.67628,8.78906,,-82.67628,8.78906,
9,AR,Argentina,AR,ARG,2766890,,,,,,,


#### Add latitude, longitude, elevation, and population data for admin1 divisions

In [19]:
admin1 = pd.read_csv(NEO4J_HOME / "import/00f-GeoNamesAdmin1.csv", dtype='str')
admin1 = pd.merge(admin1, df, on='geonameId', how='left')
admin1.fillna('', inplace=True)
admin1.to_csv(NEO4J_HOME / "import/00f-GeoNamesAdmin1.csv", index=False)

In [20]:
admin1.head()

,id,name,code,parentId,geonameId,latitude,longitude,elevation,population
0,AD.06,Sant Julia de Loria,06,AD,3039162,42.46247,1.48247,,9448
1,AD.05,Ordino,05,AD,3039676,42.59758,1.52573,,3467
2,AD.04,La Massana,04,AD,3040131,42.55417,1.48333,,8953
3,AD.03,Encamp,03,AD,3040684,42.53333,1.63333,,13685
4,AD.02,Canillo,02,AD,3041203,42.58333,1.65833,,567


#### Add latitude, longitude, elevation, and population data for admin2 divisions

In [21]:
admin2 = pd.read_csv(NEO4J_HOME / "import/00g-GeoNamesAdmin2.csv", dtype='str')
admin2 = pd.merge(admin2, df, on='geonameId', how='left')
admin2.fillna('', inplace=True)
admin2.to_csv(NEO4J_HOME / "import/00g-GeoNamesAdmin2.csv", index=False)

In [22]:
admin2.head()

,id,name,parentId,geonameId,latitude,longitude,elevation,population
0,AE.01.101,Abu Dhabi Municipality,AE.01,12047239,24.41361,54.43295,,1797294
1,AE.01.102,Al Ain Municipality,AE.01,12047240,24.15223,55.8204,,776935
2,AE.01.103,Al Dhafra,AE.01,12047241,23.65745,53.72225,,319433
3,AE.04.701,Al Fujairah Municipality,AE.04,12047242,25.13557,56.33279,,168822
4,AE.04.702,Dibba Al Fujairah Municipality,AE.04,12047243,25.5858,56.24792,,67989


#### Add latitude, longitude, elevation, and population data for cities

In [23]:
city = pd.read_csv(NEO4J_HOME / "import/00h-GeoNamesCity.csv", dtype='str')
city = pd.merge(city, df, on='geonameId', how='left')
city.fillna('', inplace=True)
city.to_csv(NEO4J_HOME / "import/00h-GeoNamesCity.csv", index=False)

In [24]:
city.head()

,id,name,parentId,geonameId,latitude,longitude,elevation,population
0,3040051,les Escaldes,AD.08,3040051,42.50729,1.53414,,15853
1,3041563,Andorra la Vella,AD.07,3041563,42.50779,1.52109,,243
2,290594,Umm Al Quwain City,AE.07,290594,25.56473,55.55517,,62747
3,291074,Ras Al Khaimah City,AE.05,291074,25.78953,55.9432,,351943
4,291580,Zayed City,AE.01.103,291580,23.65416,53.70522,,63482
